In [2]:
# Import libraries

import csv
import re
import time
import os
import sys

from tqdm import tqdm
from tqdm.notebook import trange, tqdm
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from urllib.parse import urljoin

from threading import Thread

import pandas as pd

In [3]:
base_path = os.path.join(os.path.abspath(''), 'data')

#driver = webdriver.Chrome()
driver = webdriver.Firefox()

In [6]:
driver.get('https://www.sueddeutsche.de/news?search=Russland+Russ&sort=date')

In [23]:
# Webscraping algorithm

start_date = "01.08.2022"
end_date = "01.10.2022" # nicht vergessen, dass end_date nicht mitgezählt wird
sort = "date"
search = "Russland+Russ"

not_readable = []
df = pd.DataFrame(columns=['title','author','category', 'date', 'text'])
df.index.name = 'row_id'

driver.switch_to.window(driver.window_handles[0])
driver.get(f'https://www.sueddeutsche.de/news?search={search}&sort={sort}&startDate={start_date}&endDate={end_date}')

last = driver.find_elements(By.CSS_SELECTOR, 'ul#paging li.navigation ol li')
last = last[-1].text

#hits = 5000 # max 5000 hits per search

pbar = tqdm(total=int(last)*50, position=0)
for i in range(int(last)):

    driver.switch_to.window(driver.window_handles[0])
    driver.get(f'https://www.sueddeutsche.de/news/page/{i+1}?search={search}&sort={sort}&typ[]=article&startDate={start_date}&endDate={end_date}')

    wait = WebDriverWait(driver, 10)
    element = wait.until(
        lambda d: d.find_element(By.CSS_SELECTOR,'li.whereami') and d.find_element(By.CSS_SELECTOR,'li.whereami').text == str(i+1)
        )

    time.sleep(0.5)

    links = driver.find_elements(By.CSS_SELECTOR, 'a.entrylist__link')

    for j in range(len(links)):
        pbar.update(1)
    
        driver.switch_to.window(driver.window_handles[0])

        links[j].send_keys(Keys.CONTROL + Keys.RETURN)
        driver.switch_to.window(driver.window_handles[-1])

        try:      
            wait = WebDriverWait(driver, 10) 
            wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'header span[data-manual="title"]')))

            soup = BeautifulSoup(driver.page_source, 'html.parser')

            title = soup.select('header span[data-manual="title"]')[0].text

            paywall = soup.select_one('div#sz-paywall')
            if paywall is not None:
                not_readable.append(driver.current_url)
                driver.close()
                
                continue
            
            category = str(driver.current_url).split('/')[3]

            date = soup.select_one('header time[data-manual="date"]')
            date = date["datetime"]
            
            text = soup.select('div[data-manual="body"] > p')
            text = [t.text for t in text]
            text = " ".join(text)

            author = soup.select_one('div[data-manual="author"] > a')
            if author is not None:
                author = author.text
            else:
                author = 'NA'
                
        except:
            print("Error loading page: ", driver.current_url)
            driver.close()

            continue

        df2 = pd.DataFrame({'title': title, 'author': author,'category':category, 'date': date, 'text': text},index=[0])
        df2 = df2.astype(str)
        
        df = pd.concat([df, df2], ignore_index=True)

        driver.close()
        time.sleep(0.75) # To not overload the server (hopefully)
    
pbar.close()
df.to_csv(f"{base_path}\\sz_articles_{start_date.split('.')[2]}_{start_date.split('.')[1]}_{end_date.split('.')[1]}.csv",quoting=csv.QUOTE_NONNUMERIC)

  0%|          | 0/1000 [00:00<?, ?it/s]

Error loading page:  https://sz-magazin.sueddeutsche.de/das-beste-aus-aller-welt/schneekrabben-arktis-hacke-92024?reduced=true
Error loading page:  https://sz-magazin.sueddeutsche.de/das-beste-aus-aller-welt/wutbuerger-axel-hacke-putin-91973?reduced=true


In [20]:
df

,title,author,category,date,text
0,Krieg gegen die Ukraine: So ist die Lage,NA,politik,2022-12-31 23:13:19,Kiew (dpa) - Kurz vor dem Jahreswechsel ist in...
1,Aachener Bischof fordert Freiheit für Karlspre...,NA,politik,2022-12-31 19:06:03,Aachen/Essen/Köln (dpa) - Aachens Bischof Helm...
2,Putin hält Rede bei Soldaten - und wirft Weste...,NA,politik,2022-12-31 16:12:30,Moskau (dpa) - Der russische Präsident Wladimi...
3,"Putins Mann fürs Grobe - ""Wagner""-Chef Prigoschin",NA,politik,2022-12-31 13:50:06,Moskau (dpa) – Vor dem Start ins neue Kriegsja...
4,Bovenschulte verbreitet in Neujahrsansprache Z...,NA,bayern,2022-12-31 11:13:07,Bremen (dpa/lni) - Bremens Bürgermeister Andre...
...,...,...,...,...,...
560,Lawrow verteidigt Angriffe auf ukrainische Inf...,NA,politik,2022-12-01 11:32:34,Moskau (dpa) - Russlands Außenminister Sergej ...
561,Viele ehrgeizige Missionen ins All geplant,NA,panorama,2022-12-01 11:14:58,"Washington (dpa) - Missionen zum Mond, zum Mar..."
562,Ukrainischer Kanute versteigert olympische Gol...,NA,sport,2022-12-01 09:25:38,Laguna Niguel (dpa) - Der ukrainische Kanu-Oly...
563,LNG-Terminal: Alle Pipeline-Rohre in Wilhelmsh...,NA,wirtschaft,2022-12-01 06:38:19,Wilhelmshaven (dpa/lni) - Die Erdarbeiten für ...


In [21]:
len(df) + len(not_readable)

897

In [10]:
test = df.drop_duplicates(subset=['title','date'], inplace=False)
test

,title,author,category,date,text
0,US-Regierung: Kommunikationskanäle nach China ...,NA,politik,2023-05-31 21:17:20,Washington (dpa) - Nach dem gefährlichen Vorfa...
1,Vier russische Generalkonsulate müssen schließen,NA,politik,2023-05-31 19:47:37,Berlin/Moskau (dpa) - Als Reaktion auf die Aus...
2,Darf die Ukraine russisches Territorium angrei...,NA,politik,2023-05-31 18:38:46,"An beinahe jede Waffenlieferung, die die Ukrai..."
3,Prozess: Beschwerde von Kremlgegner Nawalny ab...,NA,panorama,2023-05-31 18:31:33,Moskau (dpa) - Noch vor dem offiziellen Beginn...
4,Nachrichten am 31. Mai 2023,NA,politik,2023-05-31 17:46:17,
...,...,...,...,...,...
479,"Papst wählt in Ungarn klare Worte: ""Öffnen wir...",NA,politik,2023-05-01 12:56:48,Budapest/Rom (dpa) - Während seines dreitägige...
480,Krieg gegen die Ukraine: So ist die Lage,NA,politik,2023-05-01 11:33:11,Kiew (dpa) - Der ukrainische Präsident Wolodym...
481,"""Stolz Chinas"": Ding krönt den Schach-Masterplan",NA,sport,2023-05-01 10:02:19,Astana (dpa) - Die Erfüllung von Chinas Schach...
482,Brych verurteilt Anfeindungen gegen Kollegen S...,NA,sport,2023-05-01 08:17:26,München (dpa) - Spitzenschiedsrichter Felix Br...


In [21]:
df.to_csv(f"G:\coding\PcToLaptop\CER\project\data\\sz_articles_{start_date.split('.')[2]}_{start_date.split('.')[1]}.csv",quoting=csv.QUOTE_NONNUMERIC)